In [ ]:
import sys
sys.path.append("./../")
from evaluation.gsm8k import judge
import json
from thirdparty.openai.grade_school_math.dataset import read_jsonl
import numpy as np
from scipy.stats import bootstrap

: 

In [10]:
reference = read_jsonl("./../data/gsm8k/test.json")

In [11]:
names = ["output_mistral_instruct_v2_sft_gsm8k.json", 
         "output_mistral_instruct_v2_pause_token_sft_gsm8k.json",
         "output_mistral_instruct_v2_sft_gsm8k_updated_embedding_layer.json"]

name2data = {}
for name in names:
    with open(f"./../result/{name}", "r") as f:
        data = json.load(f)
        
    for entry in data:
        entry["answer"] = entry["output"]
    name2data[name] = data

In [12]:
for name, data in name2data.items():
    
    reference_crop = reference[:len(data)]
    results = judge(data, reference_crop)
    
    results = np.array(results).reshape((1,-1))
    
    ci = bootstrap(results, np.mean).confidence_interval
    
    print(f"{name}: {np.mean(results)}, {ci}")


output_mistral_instruct_v2_sft_gsm8k.json: 0.3564356435643564, ConfidenceInterval(low=0.26732673267326734, high=0.45544554455445546)
output_mistral_instruct_v2_pause_token_sft_gsm8k.json: 0.42574257425742573, ConfidenceInterval(low=0.33663366336633666, high=0.5247524752475248)
output_mistral_instruct_v2_sft_gsm8k_updated_embedding_layer.json: 0.40594059405940597, ConfidenceInterval(low=0.31683168316831684, high=0.504950495049505)
